In [134]:
import config
import numpy as np
import pandas as pd
import tensorflow as tf
import pickle
import config
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import initializers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Conv2D,AveragePooling2D, BatchNormalization,Dropout,Flatten,Dense
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

from sklearn.metrics import pairwise_distances_argmin_min, pairwise_distances
plt.style.use('ggplot')

import implicit
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
import io
import librosa
import librosa.display
import soundfile as sf
import glob
import scipy.sparse.sputils

import pydub
from urllib.request import urlopen
import requests
from shutil import copyfileobj
from tempfile import NamedTemporaryFile
from urllib.request import urlopen, Request
from implicit.als import AlternatingLeastSquares
from scipy.sparse import coo_matrix

# Latent Features 

These functions are to enable us to find similar songs even if they do not appear in the mixesdb dataset. This will happen quite a lot as there are many songs out there.


im in two minds about whether I should scale the data or not. Im sort of thinking if the neural net gave bigger numbers to certain features it may mean that they should have bigger influence anyway. I will try both ways to see which one produces better results, although very subjective what best results would look like.


Also I was wondering if I should use Cosine similarity vs Euclidean distance so did a bit of reading and found this article -https://cmry.github.io/notes/euclidean-v-cosine . Bascially says that if we care about magnitude we should use Euclidean, and I think we do care about magnitude.

In [135]:
client_id= config.client_id
client_secret =config.client_secret

username = '1143043561'
auth = SpotifyClientCredentials(
    client_id=client_id,
    client_secret=client_secret
)


try:
    token = auth.get_access_token()
except:
    os.remove(f'.cache-{username}')
    token = auth.get_access_token()
    
#create spotify object
spotify = spotipy.Spotify(auth=token)

In [136]:
def closest_k_nodes(node, nodes,k=5):
    node=node.reshape(1,-1)
    dist_2 = pairwise_distances(nodes,node)
    smallest_inds=np.argsort(dist_2[:,0])[:k]
    smallest_dists=dist_2[smallest_inds]
    return (smallest_inds, smallest_dists)

In [137]:
def find_similar(song_id,song_list=music_df,num_songs=5,diff_genre='no',same_key='yes',similar_tempo='yes',margin=3,scaled=False):
    
    
    song_index=song_list[song_list.id==song_id].index[0]
    
   
    orig_key = song_list.loc[song_index,'key']
    orig_tempo= song_list.loc[song_index, 'tempo']
    orig_genre= song_list.loc[song_index,'genre_approx']
    node = song_list.loc[song_index]
    
    #check if you want songs of same key
    if same_key=='yes':
    #if yes then filter out other keys    
        print(f'key:{orig_key}')
        song_list = song_list[song_list.key ==orig_key]
    #can also enter number to specify what key you want    
    elif type(same_key) !=str:
        song_list = song_list[song_list.key==same_key]


    # check if you want similar tempo    
    if similar_tempo=='yes':
        print(f'tempo:{orig_tempo}')
        #if yes can also specify how similar you want it 
        song_list=song_list[song_list.tempo.between(int(orig_tempo)-margin,int(orig_tempo)+margin)]
        
    elif type(similar_tempo) !=str:
        #can also specify a specific tempo that you want
        song_list = song_list[song_list.tempo.between(int(similar_tempo)-margin,int(similar_tempo)+margin)]
    
    #check if you want a different genre
    if diff_genre=='yes':
        print(f'genre:{orig_genre}')
        song_list= song_list[song_list.genre_approx != orig_genre]
        
    
    #adds requested song back into dataframe in case it was filtered out
    song_list=song_list.append(node,ignore_index=True)
    song_id_info=song_list.copy()
    #makes sure certain columns are ints as they come as strings from spotify
    song_list.key=song_list.key.astype(int)
    song_list['mode']=song_list['mode'].astype(int)
    song_list.time_signature=song_list.time_signature.astype(int)
    #exclude any other column which is a string
    song_list=song_list.select_dtypes(exclude=object)
    
    #scale the data
    if scaled==True:
        scaler = StandardScaler()
        song_list = scaler.fit_transform(song_list)

        node = song_list[-1]
        nodes = song_list
    
    else:
        node=np.asarray(song_list.iloc[-1])
        nodes=song_list
    
    #find closest song using euclidean distance
    closest_inds,distances = closest_k_nodes(node,nodes,num_songs)
    
    #get song ids of closest songs
    song_ids=song_id_info.loc[closest_inds,'id']
    #make sure there is no NANs 
    song_ids=[x for x in song_ids if str(x) != 'nan']
    
    #find track names using spotify API
    tracks=spotify.tracks(list(song_ids))
    song_names=[]
    for track in tracks['tracks']:
        song_names.append((track['name'],track['artists'][0]['name']))

    #create dataframe of songs and their euclidean distance from the original song
    close_song_df=pd.DataFrame({'song_ids':song_ids,
                                'song_names':song_names,
                                'distances':distances.flatten()})
    return (close_song_df)

In [138]:
def spectrogram_then_latent(url, song_id, name, model):
    '''basically the same as the function i used to
    create a load of spectrograms but just for 1 this time
    takes  url and turns the preview mp3 
        into a spectrogram and then uses a model to 
        extract latent features'''   

    if url != None:
        try:
            mp3_url = url
            wav = io.BytesIO()
            with urlopen(mp3_url) as r:
                r.seek = lambda *args: None  # allow pydub to call seek(0)
                pydub.AudioSegment.from_file(r).export(wav, "wav")

            wav.seek(0)
            y, sr = librosa.load(wav)

            # mel-scaled power (energy-squared) spectrogram
            mel_spec = librosa.feature.melspectrogram(y,
                                                      sr=sr,
                                                      n_mels=128,
                                                      hop_length=1024,
                                                      n_fft=2048)
            # Convert to log scale (dB). We'll use the peak power as reference.
            log_mel_spec = librosa.amplitude_to_db(mel_spec, ref=np.max)
            #make dimensions of the array smaller
            log_mel_spec = np.resize(log_mel_spec, (128, 644))

            log_mel_spec_arr = log_mel_spec.reshape(
                log_mel_spec.shape[0], log_mel_spec.shape[1], 1)
            pre_process = np.expand_dims(log_mel_spec_arr, axis=0)
            pre_process = pre_process / 255
            latent = model.predict(pre_process)
            

        except:
            print('song has no useable url')
            return
    
        #just going to keep it as a df for ease of use even though only 1 row
        latent_df= pd.DataFrame(latent)
        latent_df=latent_df.loc[~(latent_df==0).all(axis=1)]
        latent_df['song_names']= name
        latent_df['id']= song_id
    
    else:
        print('song has no useable url')
        return
    
    return (latent_df)

In [139]:
def get_music_quals(id_list):
    '''get music qualities such as tempo and time sig
        with a list of song ids'''
    qualities={}
    count=0
    for song in id_list:
        if type(song)==str:
            quals=spotify.audio_features(song)
            try:
                qualities[song]=quals
            except:
                print(count)
                count+=1
    return qualities

In [140]:
def get_spotify_quals(music_df):
    '''get spotify music qualities and put into
        dataframe then merge into main dataframe'''
    #get ids of songs so can search spotify
    id_list=list(music_df['id'].values)
    #earlier function that helps look for music quals
    quals = get_music_quals(id_list)
    #put the qualities as column headers
    df=pd.DataFrame(columns=list(quals[list(quals.keys())[0]][0].keys()))
    for ind, key in enumerate(quals.keys()):
    #iterate over songs and get the spotify qualities for each song into the df
        song=quals[key]
        try:
            df.loc[ind]=list(song[0].values())
        except:
            print(song)
    #merge the two dataframes together on id and return
    orig_and_spotify =pd.merge(music_df,df,on='id',how='outer')
    
    #drop columns we dont really need
    orig_and_spotify.drop(columns=['track_href','analysis_url','duration_ms','type','uri'],inplace=True)
    #turning some of the columns into ints so can be used
    #using try and except as there may be some NANs
    try:
        orig_and_spotify.key=orig_and_spotify.key.astype(int)
        orig_and_spotify['mode']=orig_and_spotify['mode'].astype(int)
        orig_and_spotify.time_signature=orig_and_spotify.time_signature.astype(int)
        return orig_and_spotify
    except:
        return orig_and_spotify
    

# Collaborative Model

This section is for when a song is found within our mixes dataset - here we are going to use alternating least squares. The biggest reason I chose this form of matrix factorization over something like logistic matrix factorization is that I want to eventually re do this project in Spark for practice and Spark also has an alternating least squares library.

In [141]:
collab_df=pd.read_csv('../song_collab_filtering/mixesdb_df_for_recs.csv')

/Users/danielchow/opt/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [142]:
#needed to make the training not take ages
os.environ["MKL_NUM_THREADS"] = "1"

In [143]:
def collab_filter(song_id, user_song_df, num_songs=5):
    '''
    song_id = spotify id for individual song
    user_song_df= dataframe with users, songs, playcounts etc
    for the time being i am not going to enable filtering by key/tempo as not enough songs
    but in future will do
    '''

    song_num = user_song_df[user_song_df.spotify_id == song_id].song_nums.values[0]
    
    #orig_key = song_list[song_list.spotify_id==song_id].key.values[0]
    #orig_tempo= song_list[song_list.spotify_id==song_id].tempo.values[0]

    #check if you want songs of same key
    #if same_key=='yes':
    #if yes then filter out other keys
    #    print(f'key:{orig_key}')
    #    song_list = song_list[song_list.key ==orig_key]

    #can also enter number to specify what key you want
    # elif type(same_key) !=str:
    #     song_list = song_list[song_list.key==same_key]

    # check if you want similar tempo
    #  if similar_tempo=='yes':
    #     print(f'tempo:{orig_tempo}')
    #if yes can also specify how similar you want it
    #     lower= int(orig_tempo)-margin
    #    higher=int(orig_tempo)+margin
    #    song_list=song_list[song_list.tempo.between(lower,higher)]

    #elif type(similar_tempo) !=str:
    #can also specify a specific tempo that you want
    #   song_list = song_list[song_list.tempo.between(int(similar_tempo)-margin,int(similar_tempo)+margin)]

    # refined_ids=song_list.spotify_id
    #this will be updated
    user_song_refined = user_song_df
    #[user_song_df.spotify_id.isin(
    #    refined_ids)].copy()

    plays = user_song_refined['plays']
    user_nums = user_song_refined.user_nums
    song_nums = user_song_refined.song_nums

    B = coo_matrix((plays, (song_nums, user_nums))).tocsr()

    model = AlternatingLeastSquares(factors=30)
    model.fit(B)
    songs_inds = model.similar_items(song_num, N=num_songs)
    songs_inds = [tup[0] for tup in songs_inds]
    filtered_df= user_song_df[user_song_df.song_nums.isin(songs_inds)]
    filtered_df.drop_duplicates(subset=['spotify_id'],inplace=True)
    return filtered_df

In [144]:
collab_filter('3SRLSSTHyRY50ufRVA8BzK',collab_df,num_songs=10)

/Users/danielchow/opt/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,set_title,song_name_mixesdb,song_artist,song_name,DJ,year,spotify_song_name,spotify_id,preview,songs,user_nums,song_nums,plays
9044,- SCAR - Data Transmission Mix Of The Day,Adam F - Metropolis,Adam F,Metropolis,SCAR,NaN,Metropolis,3SRLSSTHyRY50ufRVA8BzK,https://p.scdn.co/mp3-preview/a58883627c312ab3231e9a5c22e610454e99e435?cid=461b02dc885841c0a798981c336f46f0,Metropolis,22467,307797,1.0
60937,- North Base - Data Transmission Podcast (DTP511),Dillinja - Nasty Ways,Dillinja,Nasty Ways,North Base,NaN,Nasty Ways (feat. Lorreia Loushy),53TFvcQoCYaGytR1HezNlL,https://p.scdn.co/mp3-preview/f621430489ed18061d1892881f0658d0f5fc3e78?cid=461b02dc885841c0a798981c336f46f0,Nasty Ways,19668,572438,1.0
136406,"- High Contrast - One Mix, Beats 1",High Contrast - If We Ever,High Contrast,If We Ever,High Contrast,NaN,If We Ever - Unglued Remix,5lnCGe2IEKn1V2G3B5rzZQ,https://p.scdn.co/mp3-preview/3434c9c07616b87f6cf113bb76b46252c0cd05b6?cid=461b02dc885841c0a798981c336f46f0,If We Ever,12848,726595,2.0
181302,- GRRL - NTS Radio,Digital & Spirit - Phantom Force,Digital & Spirit,Phantom Force,GRRL,NaN,Phantom Force - Fracture's Astrophonica Edit,3dFDjqGh2GYKZCghqsq8g6,https://p.scdn.co/mp3-preview/7868d9ffc45921c6bd02db7ed7fbe9dab3914b70?cid=461b02dc885841c0a798981c336f46f0,Phantom Force,11731,570705,1.0
275422,- North Base - Data Transmission Podcast (DTP511),Splash - Babylon,Splash,Babylon,North Base,2016,Babylon - Original Mix,05KgAsHP0YmiJ0KWP6Axf0,https://p.scdn.co/mp3-preview/35889ae2b1b71bcf9606d9fb75d7acf468e75ed0?cid=461b02dc885841c0a798981c336f46f0,Babylon,19668,1236610,1.0
521031,- Karl Haze - Hibernation,Usual Suspects - Ja Bless,Usual Suspects,Ja Bless,Karl Haze,2015,-,-,-,-,15131,1366124,1.0


# Full Process

In [145]:
#load pretrained model
model = load_model('more_data_08-2.07.h5')
# take the last layer off the model so we can get to the latent features
new_model =model
new_model.layers.pop()
new_model_2 = Model(new_model.input, new_model.layers[-3].output)

In [146]:
music_df=pd.read_csv('music_values.csv')

In [147]:
music_df.head()

,0,1,10,11,12,13,14,15,16,17,18,19,2,20,21,22,23,24,25,26,27,28,29,3,30,31,32,33,34,35,36,37,38,39,4,5,6,7,8,9,acousticness,danceability,energy,genre_approx,id,instrumentalness,key,liveness,loudness,mode,song_names,speechiness,tempo,time_signature,valence
0,-10.647506,-3.830119,-2.276320,2.384080,-0.718876,-5.309031,-5.939366,-5.110894,-7.023750,-9.320401,-6.142007,-15.124358,-3.803414,3.296709,-4.180166,-5.004883,-4.009501,0.944500,-5.554684,-13.873141,-13.509689,3.275274,-10.599348,9.415700,-14.636245,4.676776,-3.126974,-7.340863,-1.744714,5.221344,-7.517174,-10.004066,-9.198299,1.302922,-13.193883,5.373672,-1.729031,-3.662412,-12.707911,2.420905,0.00624,0.671,0.936,0,0C6aMqg0IlMqSNTxpmRbko,0.910,5,0.1020,-8.993,1,Elements,0.0404,130.009,4,0.0679
1,-8.535035,-3.119350,-1.921197,2.696614,-3.005904,-2.222446,-3.111893,-1.391432,-7.305496,-5.574765,-5.882614,-10.170545,-0.255582,1.412339,-2.735173,-3.437195,-3.949102,-0.413025,-2.347446,-9.670558,-8.983416,2.528855,-8.407037,4.537114,-9.224747,3.873955,-2.383328,-1.652759,-1.037005,4.553837,-4.992689,-6.389563,-5.732815,1.308075,-10.282629,4.008411,-1.651399,-0.713237,-8.395864,2.246202,0.00284,0.645,0.947,0,6AjJ0GijsnB8woz1O4wfkq,0.839,2,0.0888,-8.520,1,Tenebrous,0.0442,129.991,4,0.3950
2,-10.982079,-4.839407,4.052264,5.559170,-2.235347,-2.095156,-0.474506,-6.318758,-4.672943,-4.355182,-3.618807,-11.790941,-4.467938,1.538228,-2.214649,-5.603659,-6.627180,3.440638,-5.447720,-9.346800,-7.070545,-3.116416,-7.463638,5.526833,-7.606748,2.062241,-4.003060,-4.367800,-4.302842,-3.939501,-7.265422,-8.715718,-2.724717,-0.757078,-8.012507,10.972013,1.473505,-2.994816,-5.296344,0.704502,0.01220,0.745,0.788,0,3UIEGb11u2Ll7wzYyC2nvi,0.885,1,0.0835,-7.438,1,19 calls,0.0414,128.005,4,0.1100
3,-16.457735,-3.879791,5.137872,6.791017,-0.217960,-4.543343,-4.776890,-10.510324,-6.026231,-7.922109,-4.683609,-21.224411,-8.015574,2.134460,-3.411675,-10.755070,-8.653569,4.135190,-7.043923,-15.846562,-14.240404,-2.654283,-11.314114,12.154058,-13.014272,4.168493,-2.543177,-8.732637,-5.558640,-1.287736,-12.466581,-14.498297,-6.117126,-2.674595,-13.356617,14.569771,-0.528583,-0.858152,-10.952185,4.132758,0.01050,0.735,0.840,0,5UduvVx5HmM4WOL0jGwdqZ,0.806,7,0.0805,-4.934,1,Dream,0.0631,126.997,4,0.3040
4,-9.155202,-2.255321,1.523480,2.419033,-1.980762,-0.339727,-0.921133,-1.097920,-6.865857,-3.310604,-4.267541,-9.240707,-2.574945,2.671082,-4.224010,-8.700339,-3.245938,1.592445,-0.251443,-7.997495,-8.892088,-0.861438,-4.643106,6.996163,-7.471015,-1.098825,-1.816144,-5.586720,-2.905124,1.846084,-5.618558,-5.818724,-4.521271,0.030358,-8.690778,9.205780,-1.224878,-0.416328,-6.897912,4.587940,0.00075,0.796,0.942,0,0P0rTrGkQMCei95omAOdQq,0.885,5,0.1010,-7.432,0,Vigilance - Original Mix,0.0645,126.996,4,0.2290


In [148]:
def get_similar_for_new(song_id,
                        collab_df,
                        music_df,
                        model,
                        num_songs=5,
                        diff_genre='no',
                        same_key='yes',
                        similar_tempo='yes',
                        margin=3,
                        scaled=False):
    '''
        First checks if song in collaborative filtering database
        if not it then checks if song id in music dataframe and 
        if song is in neither tries to create the spectrogram 
        and find spotify features for the specified song, 
        then finds similar songs to that song using the 
        aforementioned qualities
    '''

    if song_id in collab_df.spotify_id.unique():
        return collab_filter(song_id=song_id,user_song_df=collab_df,num_songs=num_songs)
    
    #checks if song in music dataframe
    if song_id in music_df.id.unique():
        #if it is in the dataframe can just find similar songs
        close_song_df = find_similar(song_id,
                                     music_df,
                                     num_songs=num_songs,
                                     diff_genre=diff_genre,
                                     same_key=same_key,
                                     similar_tempo=similar_tempo,
                                     margin=margin,
                                     scaled=scaled)

        return close_song_df

    else:
        track = spotify.track(song_id)
        song_name = track['name']
        preview_url = track['preview_url']
        #check if the song has a url
        if preview_url:
            single_song_df = spectrogram_then_latent(preview_url, song_id,
                                                     song_name, model)

            #can just use the same spotify function as before
            single_song_with_spotify = get_spotify_quals(single_song_df)

            #sometimes the names of the columns can become strings/ints so need to check their the same
            cols = [str(col) for col in list(single_song_with_spotify.columns)]

            single_song_with_spotify.columns = cols
            #need to remove genre approx in the future
            single_song_with_spotify['genre_approx'] = 10
            single_song_with_spotify = single_song_with_spotify.reindex(sorted(
                single_song_with_spotify.columns),
                                                                        axis=1)
            music_df = pd.concat([music_df, single_song_with_spotify],
                                 ignore_index=True)
            #now we have song in dataframe can just do same process as the first bit of if statement
            close_song_df = find_similar(song_id,
                                         music_df,
                                         num_songs=num_songs,
                                         diff_genre=diff_genre,
                                         same_key=same_key,
                                         similar_tempo=similar_tempo,
                                         margin=margin,
                                         scaled=scaled)
            return close_song_df

        else:
            print('song has no url/ no spotify info')
            return

In [149]:
get_similar_for_new('3SRLSSTHyRY50ufRVA8BzK',collab_df,music_df,new_model_2,same_key='yes',num_songs=10)

/Users/danielchow/opt/anaconda3/envs/tensorflow_env/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,set_title,song_name_mixesdb,song_artist,song_name,DJ,year,spotify_song_name,spotify_id,preview,songs,user_nums,song_nums,plays
9044,- SCAR - Data Transmission Mix Of The Day,Adam F - Metropolis,Adam F,Metropolis,SCAR,NaN,Metropolis,3SRLSSTHyRY50ufRVA8BzK,https://p.scdn.co/mp3-preview/a58883627c312ab3231e9a5c22e610454e99e435?cid=461b02dc885841c0a798981c336f46f0,Metropolis,22467,307797,1.0
60937,- North Base - Data Transmission Podcast (DTP511),Dillinja - Nasty Ways,Dillinja,Nasty Ways,North Base,NaN,Nasty Ways (feat. Lorreia Loushy),53TFvcQoCYaGytR1HezNlL,https://p.scdn.co/mp3-preview/f621430489ed18061d1892881f0658d0f5fc3e78?cid=461b02dc885841c0a798981c336f46f0,Nasty Ways,19668,572438,1.0
136406,"- High Contrast - One Mix, Beats 1",High Contrast - If We Ever,High Contrast,If We Ever,High Contrast,NaN,If We Ever - Unglued Remix,5lnCGe2IEKn1V2G3B5rzZQ,https://p.scdn.co/mp3-preview/3434c9c07616b87f6cf113bb76b46252c0cd05b6?cid=461b02dc885841c0a798981c336f46f0,If We Ever,12848,726595,2.0
181302,- GRRL - NTS Radio,Digital & Spirit - Phantom Force,Digital & Spirit,Phantom Force,GRRL,NaN,Phantom Force - Fracture's Astrophonica Edit,3dFDjqGh2GYKZCghqsq8g6,https://p.scdn.co/mp3-preview/7868d9ffc45921c6bd02db7ed7fbe9dab3914b70?cid=461b02dc885841c0a798981c336f46f0,Phantom Force,11731,570705,1.0
275422,- North Base - Data Transmission Podcast (DTP511),Splash - Babylon,Splash,Babylon,North Base,2016,Babylon - Original Mix,05KgAsHP0YmiJ0KWP6Axf0,https://p.scdn.co/mp3-preview/35889ae2b1b71bcf9606d9fb75d7acf468e75ed0?cid=461b02dc885841c0a798981c336f46f0,Babylon,19668,1236610,1.0
520997,- Karl Haze - Hibernation,Digital - Deadline,Digital,Deadline,Karl Haze,2015,Deadline,6jIrflsGCRIoII2zQpcRi9,https://p.scdn.co/mp3-preview/e8960e4021dcd10a9440b16eff50cc3df4c45e40?cid=461b02dc885841c0a798981c336f46f0,Deadline,15131,570811,1.0
521031,- Karl Haze - Hibernation,Usual Suspects - Ja Bless,Usual Suspects,Ja Bless,Karl Haze,2015,-,-,-,-,15131,1366124,1.0
728050,"- Annie Mac, DJ Fresh & Adam F - Mash Up",Sigma - The Jungle,Sigma,The Jungle,Annie Mac,2015,The Jungle,7a1dtM3afLrkGwYyJcb5qU,https://p.scdn.co/mp3-preview/9ff343dd3e78d15c30cf72ddbfc63a8099de2265?cid=461b02dc885841c0a798981c336f46f0,The Jungle,4466,1203480,1.0
1013023,- Koehler - Berceuse Heroique Mix,Mampi Swift - The One,Mampi Swift,The One,Koehler,2014,The One - InsideInfo Remix,3vu3Olx9f8D4S5SOLqxOkq,https://p.scdn.co/mp3-preview/dd8fe82862212f70cb4588c3e9bbdc9fb88a0a7c?cid=461b02dc885841c0a798981c336f46f0,The One,15600,910922,1.0


In [150]:
get_similar_for_new('0C6aMqg0IlMqSNTxpmRbko',collab_df,music_df,new_model_2,same_key='yes',num_songs=10)

key:5
tempo:130.009


,song_ids,song_names,distances
0,0C6aMqg0IlMqSNTxpmRbko,"(Elements, Joyhauser)",0.000000
1,0C6aMqg0IlMqSNTxpmRbko,"(Elements, Joyhauser)",0.000000
2,6aGQ7lewnNNvniGxSKGnke,"(It's Alright, Teebone)",13.541653
3,3IF4IzEA6T7ubvim04QOuQ,"(Broken Gate - Original Mix, Some Me)",14.570353
4,46JINIS1G8QiVKN3j5Jw3o,"(No Mercy - Original Mix, Dolby D)",14.645746
5,6j2ucwHvzqCFuVHTNUB5PK,"(You Get High In N.Y.C. (Full Length Album Mix), Revanche)",16.825911
6,7FNcb05eBgScWaEEvJRKiw,"(Lights On (feat. Ms Dynamite) - Single Mix, Katy B)",18.181411
7,1Oo0ip0kqpZKf1b1zhIDVc,"(Survive, Marco Faraone)",18.230298
8,6PUXm6m0e3YdpIbAfvkjwk,"(Tsunamy, Dusty Kid)",19.495494
9,0rJIvm9mfbJT12SoqRBYif,"(Talking Harder, Tonikattitude)",21.683482
